<a href="https://colab.research.google.com/github/msc-acse/acse-9-independent-research-project-OliverJBoom/blob/master/Chai_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Linking to Github

In [0]:
from getpass import getpass
import os

user = "OliverJBoom"
password = getpass('github password')
os.environ['GITHUB_AUTH'] = user + ':' + password

!rm -rf Chai_Base
!git clone https://$GITHUB_AUTH@github.com/msc-acse/acse-9-independent-research-project-OliverJBoom.git Chai_Base

github password··········
Cloning into 'Chai_Base'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 17 (delta 3), reused 12 (delta 2), pack-reused 0
Unpacking objects: 100% (17/17), done.


## Linking to Google Drive

In [2]:
from google.colab import drive
drive.mount("/content/gdrive/")
path = "/content/gdrive/My Drive/Chai/Data"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


## Loading Packages

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

## Loading Data

In [0]:
cu_lme =  pd.read_csv("/content/gdrive/My Drive/Chai/Data/CuLME.csv", index_col='date', parse_dates=['date'], dayfirst=True).sort_index(ascending=True)
cu_shfe =  pd.read_csv("/content/gdrive/My Drive/Chai/Data/CuSHFE.csv", index_col='date', parse_dates=['date'], dayfirst=True).sort_index(ascending=True)
cu_com_s =  pd.read_csv("/content/gdrive/My Drive/Chai/Data/CuCOMS.csv", index_col='date', parse_dates=['date'], dayfirst=True).sort_index(ascending=True)
cu_com_p =  pd.read_csv("/content/gdrive/My Drive/Chai/Data/CuCOMP.csv", index_col='date', parse_dates=['date'], dayfirst=True).sort_index(ascending=True)

universe_dict = {"lme":cu_lme, "shfe":cu_shfe, "com s":cu_com_s, "com p":cu_com_p}

In [0]:
def visualise_plots(universe_dict, frequency="MS"):
  """Plotting the line graph for all of the 
  instruments being inspected"""
  for df_name in universe_dict:
    df = universe_dict[df_name]
    plt.plot(df.index, df.price)
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.title(df_name)
    plt.grid()
    plt.show()
    
# visualise_plots(universe_dict)
# df.info()
# plt.hist((df.price), bins=20)
# plt.show()

## Preprocessing

In [214]:
def price_rename(universe_dict):
  """Renaming the column of the dataframe values to price"""
  for df_name in universe_dict:
    df = universe_dict[df_name]
    df = df.rename(columns={'value':"price"})
    universe_dict[df_name] = df
  return universe_dict


def clean_data(df, n_std = 20):
  """Removes any outliers that are further than a chosen
  number of standard deviations from the mean"""
  upper = df.price.mean() + n_std * (df.price.std())
  lower = df.price.mean() - n_std * (df.price.std())
  df.loc[((df.price > upper) | (df.price < lower)), 'price'] = None
  df.ffill(inplace=True)
  if df.price.isnull().sum() > 0: print("Rows removed:", df.price.isnull().sum())
  return df


def clean_dict_gen(universe_dict):
  """Returns a dictionary of cleaned dataframes"""
  cleaned_dict = {}
  print("Included Instrument:")
  for df in universe_dict:
    print(df)
    cleaned_dict[df] = clean_data(universe_dict[df])
    
  return cleaned_dict


universe_dict = clean_dict_gen(price_rename(universe_dict))

Included Instrument:
lme
shfe
com s
com p


In [0]:
df = universe_dict["lme"]  

In [0]:
def feature_spawn(df):
  """Spawsn features for each instrument
  Returns df with the following columns
  Price
  Log Returns
  EWMA 1 day
  EWMA 1 week
  EWMA 1 month
  EWMA 1 quarter
  EWMA 6 months
  EWMA 1 year
  Rolling vol 1 week
  Rolling vol 1 month
  Rolling vol 1 quarter
  """
  df["lg_return"] = np.log(df.price) - np.log(df.price.shift(1))
  #df["deciles_lg_return"] = pd.qcut(df.lg_return, 10, labels=False)
  df.lg_return.fillna(0, inplace=True)
  
  hlf_dict = {"week":5, "month":22, "quarter":66, "half_year":130, "year":261}

  for half_life in hlf_dict:
    df["ema_" + half_life] = df.price.ewm(span=hlf_dict[half_life]).mean()
    
  for i, half_life in enumerate(hlf_dict):
    if i < 3:
      df["roll_vol_" + half_life] = df.price.rolling(window=hlf_dict[half_life]).std(ddof=0)
  
  df.dropna(inplace=True)
  return df

In [0]:
df = feature_spawn(df)

In [0]:
def feature_view(df):
  """Visualises the features for an instrument"""
  fig, axarr = plt.subplots(len(df.columns), 1, figsize=(10, 4 * len(df.columns)))

  for ax, df_name in zip(axarr.flatten(), df.columns):
      ax.set_title(df_name)
      ax.plot(df.index, df[df_name])
      ax.grid()
      ax.legend()
      
  plt.show()
# feature_view(df)

## PCA

In [192]:
def dimenionality_selector(df):
  """Returns the number of dimensions that reaches the 
  threshold level of desired variance"""
  for n_dim in range(1, 11):
    pca = PCA(n_components=n_dim)
    pca.fit(df)
    if sum(pca.explained_variance_ratio_) > 0.99: 
      print("Number of dimensions:", n_dim)
      return n_dim
  print("No level of dimensionality reaches threshold variance level")
  return None

    
n_dim = dimenionality_selector(df)

Number of dimensions: 3


In [206]:
# Creating PCA object
pca = PCA(n_components=n_dim)
# Fitting and transforming PCA data
pca.fit(df)
pca_reduced = pca.transform(df) 

print("Explained Variance:", pca.explained_variance_ratio_, 
     "\nExplained Variance Sum:", sum(pca.explained_variance_ratio_))

# df_pca = pd.DataFrame(pca_reduced, index=df.index) 
# df_inverse = pd.DataFrame(pca.inverse_transform(pca_reduced), index=df.index, columns=df.columns)

Explained Variance: [0.93443161 0.05385666 0.00543059] 
Explained Variance Sum: 0.9937188687776095


In [217]:
for df_name in universe_dict:
  universe_dict[df_name] = feature_spawn(universe_dict[df_name])
  print(universe_dict[df_name].iloc[0:3])

             price  lg_return  ...  roll_vol_month  roll_vol_quarter
date                           ...                                  
2007-06-07  7506.0   0.000800  ...      293.775879        652.807421
2007-06-08  7220.0  -0.038848  ...      265.816407        632.438628
2007-06-11  7445.5   0.030755  ...      233.317850        606.059879

[3 rows x 10 columns]
              price  lg_return  ...  roll_vol_month  roll_vol_quarter
date                            ...                                  
2007-06-27  60450.0  -0.029824  ...     1276.421344       3891.090495
2007-06-28  61610.0   0.019008  ...     1329.865152       3873.242760
2007-06-29  63150.0   0.024689  ...     1330.412615       3840.450525

[3 rows x 10 columns]
             price  lg_return  ...  roll_vol_month  roll_vol_quarter
date                           ...                                  
1996-10-10  0.9320   0.017316  ...        0.018157          0.022176
1996-10-11  0.9315  -0.000537  ...        0.018738  